In [1]:
import torch
import numpy as np

# Tensor: creating and attributes
1. 新建
   - 直接指定初始值和shape，用zeros，ones，<font color=blue>rand生成[0-1)均匀分布</font>
   - 别的数据类型list, tuple, numpy转变：torch.tensor([[1, 2],[3, 4]]), torch.from_numpy(np_array)
   - 用别的tensor的shape：torch.ones_like(tensor_x), torch.rand_like(tensor_x, dtype=torch.float)
2. 属性: shape, dtype, device, layout, memory_format

## 1. 新建Tensor

### 1.1 指定初始值和shape新建tensor
1. ones, zeros, empty等操作和numpy相似
   - shape可以是tuple，也可以直接给各个dimension的大小，而不package到tuple中
2. rand生成[0-1)均匀分布
   - 可以用manual_seed()来设定随机种子

In [18]:
shape = (2, 3)
x = torch.zeros(shape)
y = torch.ones((2, 3), dtype=torch.int16) # 可以指定数据类型
x, y

(tensor([[0., 0., 0.],
         [0., 0., 0.]]),
 tensor([[1, 1, 1],
         [1, 1, 1]], dtype=torch.int16))

In [3]:
# random seed
torch.manual_seed(1729)  # 设定随机种子
r1 = torch.rand(shape)   # [0, 1)均匀分布
r2 = torch.rand(shape)

torch.manual_seed(1729)
r3 = torch.rand(shape) 
r1, r3 == r1

(tensor([[0.3126, 0.3791, 0.3087],
         [0.0736, 0.4216, 0.0691]]),
 tensor([[True, True, True],
         [True, True, True]]))

### 1.2 将python list，tuple或numpy array转变成tensor类型

In [4]:
# from list
data = [[1, 2],[3, 4]]
x = torch.tensor(data)
x, x.dtype

(tensor([[1, 2],
         [3, 4]]),
 torch.int64)

In [5]:
# from ndarray
a = np.array(data)
x_np = torch.tensor(a)
y_np = torch.from_numpy(a)
x_np == y_np

tensor([[True, True],
        [True, True]])

In [6]:
# from混合类型，会自动做datatype cast
data = [[1, 2],(3.5, 4)]
x_data = torch.tensor(data)
x_data, x_data.dtype

(tensor([[1.0000, 2.0000],
         [3.5000, 4.0000]]),
 torch.float32)

In [7]:
# tensor转变成numpy array
tensor_x = torch.rand(2, 3)
numpy_x = tensor_x.numpy()
tensor_x, numpy_x

(tensor([[0.2332, 0.4047, 0.2162],
         [0.9927, 0.4128, 0.5938]]),
 array([[0.23321933, 0.40465623, 0.2162376 ],
        [0.99269456, 0.41275233, 0.59382254]], dtype=float32))

### 1.3 用已有tensor的shape和datatype来新建tensor
这里要注意数据类型变化

In [8]:
y_ones = torch.ones_like(x_data, dtype=torch.int16)

# y_rand = torch.rand_like(y_ones) # Error，x_data原来的value是int，数据类型冲突
y_rand = torch.rand_like(y_ones, dtype=torch.float)
y_ones, y_rand

(tensor([[1, 1],
         [1, 1]], dtype=torch.int16),
 tensor([[0.6128, 0.1519],
         [0.0453, 0.5035]]))

## 2. tensor的属性
 - t.shape
 - t.dtype：新建tensor的默认类型是float32
 - t.device：存储位置在CPU还是GPU
 - t.layout: 说明tensor的memory layout，目前是beta功能
 - t.memory_format: 返回tensor存储的方式。有两种存储方式，用于dense tensor的torch.stride和用于sparse tensor的torch.sparse_coo.

### 2.1 数据shape和类型

In [21]:
# 设置数据类型的两种方式
a = torch.ones((2, 3), dtype=torch.int16)
c = a.to(torch.float32)
a.shape == a.size(), c.dtype # size()是method，shape是attribute

(True, torch.float32)

### 2.2 存储位置
1. tensor默认在cpu上创建，可以用tensor.to('cuda')将tensor转移到gpu上，或者直接在gpu上新建tensor
2. 一个expression中的运算对象要在同一个device上，cpu和gpu上的数据无法在一个expression中处理
3. cpu上的tensor可以跟numpy共享底层的memory location，但gpu上的tensor不行。因为gpu上只能放tensor，不能放numpy数据类型
4. 可以用context manager在不改变默认device的情况下，临时改变新建tensor的device属性
- <font color=red>注，cpu和gpu之间的数据迁移很耗费时间。</font>

In [10]:
## 在gpu上新建tensor
x = torch.rand(2, 2, device='cuda')
x

tensor([[0.3344, 0.2640],
        [0.2119, 0.0582]], device='cuda:0')

In [11]:
## 将现有tensor移动到gpu
x = torch.ones(2, 3)
if torch.cuda.is_available():
    y = x.to('cuda')
print(x.shape, x.dtype, x.device)
print(y.shape, y.dtype, y.device)

# mul = tensor * tensor2 # 错，他们一个在cpu上，一个在gpu上

torch.Size([2, 3]) torch.float32 cpu
torch.Size([2, 3]) torch.float32 cuda:0


In [12]:
## cpu上的tensor可以跟numpy共享memory location。改变其中一个的值，另一个也变
t = torch.ones(5)
n = t.numpy()
print("改变前：", t, n)
t.add_(2)
print("改变tensor，numpy对象也变：", t, n)
np.add(n, 2, out=n) # 不能用n.add(2)，因为ndarray没有add method
print("改变numpy对象后，tensor也变：", t, n)

改变前： tensor([1., 1., 1., 1., 1.]) [1. 1. 1. 1. 1.]
改变tensor，numpy对象也变： tensor([3., 3., 3., 3., 3.]) [3. 3. 3. 3. 3.]
改变numpy对象后，tensor也变： tensor([5., 5., 5., 5., 5.]) [5. 5. 5. 5. 5.]


In [13]:
## 但在gpu上无法做numpy类型对象的计算
if torch.cuda.is_available():
    t = t.to('cuda')
#     n = n.to('cuda') # numpy对象不能移到gpu上，因为没有.to method
print(t)
# n = t.numpy() # 报错：can't convert cuda:0 device type tensor to numpy.

tensor([5., 5., 5., 5., 5.], device='cuda:0')


In [24]:
# 默认device不变，临时改变device
with torch.device('cuda'):
    temp_t = torch.randn(3)
temp_t.device

device(type='cuda', index=0)

### 2.3 layout
torch.layout是一个object，它返回tensor存储的方式。有两种存储方式：
 1. 用于sparse tensor的torch.sparse_coo.还是beta功能。
 2. 用于dense tensor的<font color=blue>**torch.strided**</font>。一般的tensor都是用这种layout类型。每个strided tensor都有一个与之关联的<font color=blue>torch.Storage</font>用于存储tensor的底层data，相当于numpy中的data buffer。而tensor本身就是它的一个View。
    - 用t.stride()返回tensor t呈现View的stride信息。
    - 返回的stride信息是一个int list，每个int代表对应维度上t从一个element到下一个element中间间隔的stride长度。

In [27]:
x = torch.tensor([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]])
x.layout, x.stride() # dim1上，第1个到第2个element间隔5个值；dim2上间隔1个值

(torch.strided, (5, 1))

### 2.4 memory_format
torch.memory_format是一个object，代表tensor分配的存储格式。可能有4种类型：
1. <font color=blue>**torch.contiguous_format**</font>: Tensor分配的是dense non-overlapping memory. Strides represented by values in decreasing order.
2. <font color=blue>**torch.channels_last**</font>: Tensor分配的是dense non-overlapping memory. Strides represented by values in strides[0] > strides[2] > strides[3] > strides[1] == 1 aka NHWC order.
3. <font color=blue>**torch.channels_last_3d**</font>: Tensor分配的是dense non-overlapping memory. Strides represented by values in strides[0] > strides[2] > strides[3] > strides[4] > strides[1] == 1 aka NDHWC order.
4. <font color=blue>**torch.preserve_format**</font>: 用在类似clone等函数中，用于保留input tensor的memory format。
   - 如果input tensor分配的是dense non-overlapping memory,则output tensor strides会复制input的strides. 
   - 否则，output strides用torch.contiguous_format